In [1]:
# autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import pathmagic
from tools.project import proj_dir

In [2]:
from torch import nn

from model_setup.train_setup_ddff_mod import \
    TrainSetupDDFFBlender, TrainSetupDDFFRNN, TrainSetupDenseNet3DAutoEncoder, \
    TrainSetupDepth, TrainSetupCoCFgbgNet, TrainSetupSignedCoC, TrainSetupCoCDepthNet, TrainSetupCoC

from net.modules import AE2Decoder, OutputSplitModule
from net.consec_net import ConsecNet, PreDepthConsecNet
from net.pool_net import PoolNet, PoolNetEncoder, PoolNetDecoder
from net.vgg import VGGAE
from net.ae_rnn import AERNN
from net.unet import UNetEncoder, UNetDecoder, UNet
from net.recurrent_ae import RecurrentAE, \
    BidirRecurrentComposeAll, BidirRecurrentComposeFirstLast, BidirRecurrentComposeCenter
from net.ddff_net_har import DDFFNetHar

In [5]:
import numpy as np
from tools.tools import deterministic
from net.extend_data_modules import ExtendDataFocusDist, ExtendDataConstant, ExtendDataFov, ExtendDataMultiple
from net.ddff_net import DDFFNet

deterministic()

"""
setup = TrainSetupDepth(
    dataset="s7_5ramp",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=8, mask_loss_type=True,
    select_focus_dists=[0.1, 0.1875, 0.275, 0.3625, 0.45],
    # fixed_ramp_idx=0,
    fixed_frame_indices=range(0, 25, 6),
    model_gen=lambda: RecurrentAE(enc_sizes=[1, 1, 1, 1])
)
"""

"""
setup = TrainSetupCoCDepthNet(
    #dataset="s7_5ramp",
    dataset="s7_5ramp",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=4, mask_loss_type=True,
    select_focus_dists=[0.1, 0.1875, 0.275, 0.3625, 0.45],
    # fixed_ramp_idx=0,
    extend_data_module=ExtendDataConstant([
        "aperture", "f_number"
        #"focal_length", "sensor_size_x", "sensor_size_y", "fov_x", "fov_y", "foc_dist"
    ]),
    model_gen=lambda: OutputSplitModule(RecurrentAE, in_channels=4, enc_sizes=[1, 1, 1, 1])
)
"""

"""
setup = TrainSetupCoCDepthNet(
    #dataset="s7_5ramp",
    dataset="s7_randfoc",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=3, mask_loss_type=True,
    select_focus_dists=[0.1, 0.1875, 0.275, 0.3625, 0.45],
    # fixed_ramp_idx=0,
    extend_data_module=ExtendDataFov(use_fov_x=True, use_fov_y=True),
    model_gen=lambda: OutputSplitModule(RecurrentAE, in_channels=5, enc_sizes=[1, 1, 1, 1])
)
"""

"""
setup = TrainSetupDepth(
    dataset="s7_5ramp",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=8, mask_loss_type=True,
    select_focus_dists=[0.1, 0.1875, 0.275, 0.3625, 0.45],
    # fixed_ramp_idx=0,
    fixed_frame_indices=range(0, 25, 6),
    extend_data_module=ExtendDataFov(use_fov_x=True, use_fov_y=True), extend_encoding=True,
    model_gen=lambda: RecurrentAE(enc_sizes=[1, 1, 1, 1], dec_add_input=2)
)
"""

"""
setup = TrainSetupCoCDepthNet(
    dataset="s7_5ramp",
    #dataset="s7_randfoc",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=3, mask_loss_type=True,
    select_rel_indices=np.linspace(0, 1, 4),
    extend_data_module=ExtendDataFov(use_fov_x=True, use_fov_y=True), extend_encoding=True,
    model_gen=lambda: AE2Decoder(RecurrentAE, in_channels=3, enc_sizes=[1, 1, 1, 1], dec_add_input=2)
)
"""

"""
setup = TrainSetupDepth(
    #dataset="s7_5ramp",
    dataset="s7_randfoc",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=4, mask_loss_type=True,
    select_rel_indices=np.linspace(0, 1, 4),
    extend_data_module=ExtendDataMultiple([
        ExtendDataFov(use_fov_x=True, use_fov_y=True),
        ExtendDataConstant(["foc_dist"])
    ]),
    model_gen=lambda: RecurrentAE(enc_sizes=[1, 1, 1, 1], in_channels=6)
)
"""

"""
setup = TrainSetupDepth(
    #dataset="s7_5ramp",
    dataset="s7_randfoc",
    checkpoint_freq=5, log_img_freq=1, save_imgs=False,
    lr=1e-4, batch_size=4, mask_loss_type=True,
    select_rel_indices=np.linspace(0, 1, 4),
    model_gen=lambda: DDFFNetHar(focal_stack_size=4)
)
"""

setup = TrainSetupCoCDepthNet(
        dataset=["s7_5ramp"], mask_loss_type=True,
        select_focus_dists=[0.1, 0.1875, 0.275, 0.45],
        model_gen=lambda: AERNN(
            *DDFFNet.create_encoder_decoder(two_dec=True),
            fs_size=4, rnn_num_layers=2,
            rnn_embedding_type="stack", ccx_reduce_mode="ccx_last"
        )
    )


setup.create_components(mode="train")

setup.num_epochs = 3
setup.train(create_comp=False)

Seeeeeeeeeeeeeding!!!
Loaded pretrained
Encoder encoding dim [512, 8, 8]
Output mode: last
include_all_coc False
Val losses [Masked(MSELoss()), Masked(AccuracyLoss())]
../../../logs/tblogs/mNone_None
CREATED SummaryWriter
Train loss MultiMaskedLoss(CoCDepthLoss(
  (mse_loss): MSELoss()
))
train
AERNN [Not frozen]
Dataset type: train

Data dirs ['../../../datasets/s7_5ramp/train/seq0', '../../../datasets/s7_5ramp/train/seq26', '../../../datasets/s7_5ramp/train/seq30']
Ext tif
Multi ramp
Ramps in dataset 5
Data class <class 'data.video_depth_focus_data.VideoDepthFocusData'>
Using stats: ../../../datasets/s7_5ramp/stats_total.json
Data Stats {'focus_min': 0.1, 'focus_max': 0.5, 'frames': {'depth': {'min': 0, 'max': 1.5}}}
Using hardcoded min depth 0.03!!!
depth_normalize: Normalize(0.03, 1.5, clamp_max=False)
coc_normalize: Normalize(0, 0.00010109296328134595, clamp_max=False)
signed_coc_normalize: None
[0 3] 3
Dataset type: val

Data dirs ['../../../datasets/s7_5ramp/train/seq30']
Ext ti

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 1.95 GiB total capacity; 1.38 GiB already allocated; 17.88 MiB free; 10.57 MiB cached)

In [ ]:
from tools.tools import model_freeze_state_repr

model = setup.model
print(model_freeze_state_repr(model))